# Calculate a 2D emissions grid incorporating COVID-19 effects

In [1]:
import itertools
import netCDF4 
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt

In [2]:
input_co2_mole = "../input/mole-fraction-of-carbon-dioxide-in-air_input4MIPs_GHGConcentrations_ScenarioMIP_UoM-MESSAGE-GLOBIOM-ssp245-1-2-0_gr1-GMNHSH_2015-2500.nc"
input_co2_air = "../input/CO2-em-AIR-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_nox = "../input/NOx-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_blip = "../input/Robin_sectors_V3.csv"
convert_country_code_file = "../input/convertCountryCodes.csv"

## Collect and clean the data

In [3]:
co2_0_air = netCDF4.Dataset(input_co2_air, "r", format="NETCDF4")
nox_0 = netCDF4.Dataset(input_nox, "r", format="NETCDF4")
blip_factors = pd.read_csv(input_blip)
convert_countries = pd.read_csv(convert_country_code_file, keep_default_na=False, na_values=['_'])

In [4]:
for dimobj in nox_0.dimensions.values():
...     print(dimobj)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 360
<class 'netCDF4._netCDF4.Dimension'>: name = 'sector', size = 8
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 120
<class 'netCDF4._netCDF4.Dimension'>: name = 'bound', size = 2


In [5]:
sectors = nox_0.variables["sector"][:]
nox_0.variables["sector"]

<class 'netCDF4._netCDF4.Variable'>
int32 sector(sector)
    long_name: sector
    bounds: sector_bnds
    ids: 0: Agriculture; 1: Energy; 2: Industrial; 3: Transportation; 4: Residential, Commercial, Other; 5: Solvents production and application; 6: Waste; 7: International Shipping
unlimited dimensions: 
current shape = (8,)
filling on, default _FillValue of -2147483647 used

In [6]:
blip_factors = blip_factors[~blip_factors["1"].isna()]
blip_sectors = blip_factors["Sector"].unique()

## Perform the sector weighting


In [7]:
blip_sectors

array(['surface-transport', 'residential', 'public/commercial',
       'industry', 'international-shipping', 'international-aviation',
       'domestic-aviation', 'power'], dtype=object)

The set of sectors in our blip need to be converted into our sectors in the netCDF case. This uses:
0: Agriculture; 1: Energy; 2: Industrial; 3: Transportation; 4: Residential, Commercial, Other; 5: Solvents production and application; 6: Waste; 7: International Shipping

In [8]:
sector_dict = {"surface-transport": 3, "residential": 4, "public/commercial": -4, "industry": 2, 
 "international-shipping":7, "international-aviation": -1, "domestic-aviation": -2, "power": 1}

In [9]:
blip_factors_multi = blip_factors.copy()
blip_factors_multi.drop(["Country", "Base(MtCO2/day)", "Unnamed: 0"], axis=1, inplace=True)
blip_factors_multi["Sector"] = [sector_dict[sect] for sect in blip_factors_multi["Sector"]]
blip_factors_multi.head()

,ISO_A3,Sector,Base%,1,2,3,4,5,6,7,...,357,358,359,360,361,362,363,364,365,366
0,ALB,3,42.007797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,DZA,3,31.360946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000
2,AGO,3,29.972754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429
3,ARG,3,23.181375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951
4,ARM,3,30.523255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
blip_factors_multi.set_index(blip_factors_multi.columns[:2].to_list(), drop=True, inplace=True)
blip_factors_multi.head()

,,Base%,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
ISO_A3,Sector,,,,,,,,,,,,,,,,,,,,,
ALB,3,42.007797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DZA,3,31.360946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000,-0.407000
AGO,3,29.972754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429,-0.394429
ARG,3,23.181375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951,-0.447951
ARM,3,30.523255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
# We want to average the two sets of sector 4 together in the right ratio
for country in blip_factors_multi.index.get_level_values("ISO_A3").unique():
    if (country, 4) in blip_factors_multi.index and (country, -4) in blip_factors_multi.index:
        blip_factors_multi.loc[country, 4] = (
            blip_factors_multi.loc[country, 4].values * 
            blip_factors_multi["Base%"][country, 4] + 
            blip_factors_multi.loc[country, -4].values * blip_factors_multi["Base%"][country, -4]
            ) / (
                blip_factors_multi["Base%"][country, 4] + blip_factors_multi["Base%"][country, -4]
            )
        blip_factors_multi["Base%"][country, 4] = blip_factors_multi["Base%"][country, 4] + \
            blip_factors_multi["Base%"][country, -4]
        blip_factors_multi.drop((country, -4), inplace=True)
    elif (country, -4) in blip_factors_multi.index:
        blip_factors_multi.loc[country, 4] = blip_factors_multi.loc[country, -4]
    elif (country, 4) in blip_factors_multi.index:
        continue
    else:
        print("no data for {}".format(country))

In [12]:
# Test that this produces the right answers
example_factor = blip_factors[
    (blip_factors["ISO_A3"] == "GBR") & (blip_factors["Sector"].isin(["residential", "public/commercial"]))
][["Base%", "100"]]
assert np.isclose(blip_factors_multi.loc["GBR", 4][100], sum(
    example_factor["Base%"] * example_factor["100"]) / sum(example_factor["Base%"])
)

## Derive country and date relation
We need to assign each lat/long a country. This is slightly complicated by the country index being 2 letters in the inverse geocoder but 3 letters in our data.

In [13]:
lat, lon = nox_0.variables["lat"][:], co2_0_air.variables["lon"][:]

In [14]:
convert_countries_dict = {convert_countries["A2 (ISO)"][i]: convert_countries["A3 (UN)"][i] for i in convert_countries.index}
coords = []
lon_length = len(lon)
for latperm in lat:
    coords = coords + list(zip([latperm] * lon_length, lon))

In [15]:
results = rg.search(coords)

Loading formatted geocoded file...


In [16]:
lat_countries_dict = {coords[i]: convert_countries_dict[results[i]["cc"]] for i in range(len(coords)) 
                      if results[i]["cc"] in convert_countries_dict.keys()}

Now we must relate the dates. blip_factors uses days from 2020-01-01, and has values for every day. The netCDFs use days since 2015-01-01, which is 5 * 365 + 1 days later and monthly. 

In [17]:
date_dif = 5 * 365 + 1
date_dif

1826

In [18]:
netCDF_times = nox_0.variables["time"][:]
netCDF_tseries = pd.Series(netCDF_times)
bliptimes = blip_factors_multi.columns[blip_factors_multi.columns != "Base%"]
bliptimes = pd.Series(pd.to_numeric(bliptimes))

In [19]:
time_dict = {}
remaining_times = bliptimes.copy()
mappable_times = netCDF_tseries[(netCDF_tseries > date_dif) & (netCDF_tseries < date_dif + max(bliptimes))]
for t in mappable_times.index[:-1]:
    closeTimes = [bliptime for bliptime in remaining_times if (
        0.5 * (mappable_times[t + 1] + mappable_times[t]) - date_dif > bliptime
    )]
    time_dict[mappable_times[t]] = closeTimes
    remaining_times = remaining_times[~remaining_times.isin(closeTimes)]
time_dict[mappable_times.iloc[-1]] = list(remaining_times)

In [20]:
blip_factors_av = pd.DataFrame(index=blip_factors_multi.index, columns=time_dict.keys())
for key, val in time_dict.items():
    blip_factors_av[key] = blip_factors_multi[list(str(v) for v in val)].mean(axis=1)
    #blip_factors_av = blip_factors_multi

## Perform the emissions blip
We now have a mapping between times 

## SANDBOX